## [BASEのWeb API](https://github.com/baseinc/api-docs)の試用

アクセストークン取得のため、[こちらから](https://developers.thebase.in/)あらかじめ申請をする必要がある。
[利用規約](https://thebase.in/pages/api_term)に同意する必要がある。注意を引いた項目は以下の２つくらい

1. 自社のサービスでBASEへのリンクを貼って「使ってますよ」という事を宣伝する必要がある。
2. このAPIをBASEと競合するサービスのために使ってはならない。

ここで審査待ちが発生する。

pythonの[requests-oauthlibライブラリのOAuth2用公式ドキュメント](http://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html)を参考にする。


In [1]:
import requests
# from urllib.parse import
import webbrowser
from requests_oauthlib import OAuth1Session
from IPython.core.display import display, HTML

In [ ]:
baseurl = "https://api.thebase.in/"
client_id = ""
secret = ""

requests.get("https://api.github.com/user")

## マネーフォワード

SBI、MIZUHO、三井住友銀行といったサービスと連携しており、国内では最多らしい。
これら既存金融機関との連携にあたっては参照系API(GET)だけではなく更新系(POST)も最近始めたらしく、強い。

公開しているのはごく一部らしく、現時点では提携しないと使えない物が多い。
また、法人側(MFクラウド会計)はまだAPIを公開していない。
個人側(MFクラウド確定申告)は使えるようだが、とりあえず後回し

In [ ]:
url = "https://api.moneyforward.com/oauth/authorize"
res = requests.get(url)
print(res)

## 住信SBIネット銀行

APIの公開にあたって、IBMのサービスを利用している。Bluemixを使用しているのかと思ったが、オンプレミスでも使えるサービスらしい

口座開設待ち中

![](http://special.nikkeibp.co.jp/atcl/NBO/16/ibm0615/img1.png)

## [freee](https://secure.freee.co.jp/developers/api/doc?q=), [Zaim](https://dev.zaim.net/)

freeeのAPIはすでに[SHARESというサードパーティ企業によって利用されている](https://iotnews.jp/archives/11488)

OAuth2.0を使用

> (コールバックURIは)ローカル環境でのテストには `urn:ietf:wg:oauth:2.0:oob` をご利用ください。

とのこと


In [2]:
def call_freee_api(relative_uri):
    url = "https://api.freee.co.jp"
    appid = "2e3a35b132e32628f617a197255e523f9cc55c18032f216c32c7470e001e2653"
    secret = "d1de57fabbfb27bfa998dcb0f7bab0431de79d44c57a3823091f218bd2213b12"
    # auth_url = "https://secure.freee.co.jp/oauth/authorize"
    user_url = url + relative_uri
    params = {
        "client_id" : appid,
        "redirect_uri" : r"urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob",
        "response_type": "code", # jsonを試したが、駄目だった。必ずhtmlで帰ってくる
        "client_secret": secret
    }
    res = requests.get(user_url, params = params)
    res.raise_for_status()
    return res
    
res = call_freee_api("/api/1/users/me.json")
print(res)
# display(HTML(res.text)) # 認証用widgetが表示されるが、手元にwebサーバーがないと動かない。
with open("./freee_return.html", "w") as outfh:
    outfh.write(res.text)

HTTPError: 401 Client Error: Unauthorized for url: https://api.freee.co.jp/api/1/users/me.json?response_type=code&client_secret=d1de57fabbfb27bfa998dcb0f7bab0431de79d44c57a3823091f218bd2213b12&redirect_uri=urn%253Aietf%253Awg%253Aoauth%253A2.0%253Aoob&client_id=2e3a35b132e32628f617a197255e523f9cc55c18032f216c32c7470e001e2653

## [ZEUS](http://www.cardservice.co.jp/point/flexible_02.html)

決済サービス(正確には決済代行サービス？)
SBIの決算報告書に乗っていたから試そうかと思ったが、決済サービスなので後回しにする。